In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np
import time
import os
import sys
import time
batch_size = 512
custom_hw = Overlay("final.bit")
mnist = np.load("mnist-original.npy", allow_pickle= True)

print(mnist.item().get("data"))
dma_dataout = custom_hw.axi_dma_1
dma_datain1 = custom_hw.axi_dma_0

in1Buffer = allocate(shape=(batch_size * 25, ),  dtype=np.uint32)
outBuffer = allocate(shape=(batch_size * 10, ),  dtype=np.int32)
weights = (1 << np.arange(32, dtype=np.uint32))
pad_zero = np.zeros((batch_size, 16), dtype=np.uint32)
X = mnist.item().get("data").T
y = mnist.item().get("label")[0]

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [2]:
def pack_to_uint32_with_padding(x: np.ndarray) -> np.ndarray:

    bits_per_img = x.reshape(batch_size, 784)  # shape = (num_images, 784)

    bits_padded = np.concatenate([bits_per_img, pad_zero], axis=1)  # shape = (num_images,800)
    grouped = bits_padded.reshape(-1, 32)  # shape = (num_images * 25, 32)
    
    packed = np.sum(grouped * weights, axis=1)

    return packed

In [3]:
def feed_foward(X0):
    X0 = (X0 >> 7)
    packed = pack_to_uint32_with_padding(X0)
    np.copyto(in1Buffer, packed)
    dma_datain1.sendchannel.transfer(in1Buffer)
    dma_dataout.recvchannel.transfer(outBuffer)
    dma_dataout.recvchannel.wait() 
    return outBuffer.reshape(batch_size, 10)

In [6]:
prediction = []
start_time = time.time()
for idx in range(len(X)//batch_size):
    xs = X[batch_size * idx:batch_size * idx + batch_size]
    ys = y[batch_size * idx:batch_size * idx + batch_size]
    outputs = feed_foward(xs)
    for output, yk in zip(outputs, ys):
        prediction.append(np.argmax(output) == (yk))
end_time = time.time()
score = np.mean(prediction) * 100
print(end_time - start_time)
print(score)


25.94298481941223
96.55618106617648
